# Data Preprocessing

* `preprocess.py` dosyasının oluşturulması
* Sınıfın ve fonksiyonların oluşturulması

  * single aykırı değişken analizi → IQR

    * sil veya doldur
  * multi aykırı değişken analizi → LOF

    * sil veya doldur

---

#### Bilgi 1:

Gradient Descent optimizasyon algoritmasını kullanan modellerde **scaling** etmeliyiz.
Linear regresyon gibi algoritmalar scaling'den etkilenmediği için her koşulda scaling fonksiyonu yazabiliriz.
**Scale etmekten zarar görmezsin ama unutursan yandın.**
↳ katsayı yorumlama zorlaşabilir.

---

#### Bilgi 2:

Label Encoding, kategorik değişkenlerin ordinal sırasına göre encode etmeyebilir.
Verilen kategorik değişken elemanlarını önce harf sırasına göre sıralar sonra sayı verir.
Bunun için **Ordinal Encoding** kullanılmalıdır.

---

#### Bilgi 3:

`pandas.get_dummies()` dataframe üzerinde doğrudan çalışır ve statik bir dönüşüm yapar.
**OneHotEncoder** ise pipeline'a uyumlu çalışır ve sorun çıkarmaz.

---

#### Bilgi 4:

`.yaml` dosyaları **environment** oluştururken işe yarar.
Oluşturduktan sonra bir katkısı yoktur.

---

İstersen Markdown, LaTeX veya PDF şeklinde de düzenleyebilirim. Nasıl istersin?


---

## IQR yöntemi ile outlier tespiti ve ortalama ile doldurma

In [1]:
import seaborn as sns
import pandas as pd
df = sns.load_dataset("diamonds")
df = df.select_dtypes(include=["float64", "int64"])
df = df.dropna()
df.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [2]:
df_table = df["table"]

In [3]:
Q1 = df_table.quantile(0.25)
Q3 = df_table.quantile(0.75)
IQR = 1.5 * (Q3 - Q1)

In [4]:
lower_limit = Q1 - IQR

In [5]:
upper_limit = Q3 + IQR

In [6]:
aykiri_tf = (df_table < lower_limit) | (df_table > upper_limit)

In [8]:
df_table[aykiri_tf] = df_table.mean()

C:\Users\aalperen.arda\AppData\Local\Temp\ipykernel_7436\3741757661.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_table[aykiri_tf] = df_table.mean()


In [9]:
df_table[aykiri_tf]

2        57.457184
91       57.457184
145      57.457184
219      57.457184
227      57.457184
           ...    
53695    57.457184
53697    57.457184
53756    57.457184
53757    57.457184
53785    57.457184
Name: table, Length: 605, dtype: float64

---

## LOF ile Multi variable outlier analysis

In [10]:
import seaborn as sns
diamonds = sns.load_dataset("diamonds")
diamonds = diamonds.select_dtypes(include=["float64", "int64"])
df = diamonds.copy()
df = df.dropna()
df.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [11]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1) # contamination = kirlenme, bulastirma

In [12]:
clf.fit_predict(df)

array([-1, -1, -1, ...,  1,  1,  1])

In [13]:
df_scores = clf.negative_outlier_factor_

In [14]:
np.sort(df_scores)[0:20]

array([-8.60430658, -8.20889984, -5.86084355, -4.98415175, -4.81502092,
       -4.81502092, -4.61522833, -4.37081214, -4.29842288, -4.10492387,
       -4.0566648 , -4.01831733, -3.94882806, -3.82378797, -3.80135297,
       -3.75680919, -3.65947378, -3.59249261, -3.55564138, -3.47157375])

In [15]:
esik_deger = np.sort(df_scores)[13] 
esik_deger   # Bu degeri esik deger secmis olduk. Bundan kucuk olan degerler aykiri degerlerdir

np.float64(-3.823787967755565)

In [16]:
# Silme yontemi
new_df = df[df_scores > esik_deger]
new_df
# Bu dataframe ustten ve alttan temiz bir df dir. Cunku LOF algoritması skor hesapladigi icin belli bir noktadan uzak olan skorları dusuruyor.

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74


### Supression Method biraz daha zor ona 4. nottan bakabilirsin.